# The Mapinator Classification Part 2 (Estimation)

## The Algorithm

### ECON629C Final Draft

James Yu, 21 April 2024

Thank you to Hiro Kasahara and students in ECON629C 2023W2 at UBC for helpful feedback, comments and suggestions.

In [1]:
using Pkg
Pkg.activate("../")
#for pkg in ["HTTP", "JSON", "Distributions", "Random", "PrettyTables"]
#    Pkg.add(pkg)
#end
Pkg.instantiate()

  Activating project at `C:\Users\jbrig\Documents\research\mapinator`


## Introduction

This notebook executes the tier classification estimation required by The Mapinator Classification (Peters and Yu, 2023, work in progress). Various components of this notebook can be altered to create different estimations under different constraints. Parts of this notebook are based on my original [estimate_model.ipynb](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_model.ipynb) demo, which I put together based on previous code by myself and others who have worked on the project.

## Motivation

Creating realistic models of matching markets allows us to make accurate performance assessments of these markets. These assessments allow us to answer interesting questions about market participants and how they match, such as whether matches satisfy participants' preferences and whether matches are affected by systemic discrimination. By analyzing how participant welfare depends on market design, we can then see whether these markets and their outcomes can be improved.

Bridging the gap between theory and data is crucial for creating realistic models. In existing theory, models of matching markets focus on participants who have types representing participant characteristics (Chade et al., 2017). These include discrete high/low qualities e.g. $\theta \in \{H, L\}$, or continuous qualities e.g. $\theta \in [0, 1]$. While these types may be observable by market participants, the econometrician or theorist studying these markets in practice may not be able to observe these types. Moreover, types used by real-world market participants may not reflect the more abstract examples above studied in previous literature.

In the overall Mapinator project, we study the job market for graduates of economics Ph.D. programs. This market is a systematic, centralized process that has a structured timeline: Ph.D. students apply to jobs in the Fall, employers make offers to students in January and February, and a "Secondary Scramble" takes place in March where those who didn't match in January or February match together. There is an extensive placements dataset available for this market from the Mapinator (https://sage.microeconomics.ca/dash); while participant types and wages are not observable directly in this market, the outcomes of successful graduates are observable. Given this dataset, I focus on the question of whether we can recover candidates' and employers' unobservable types from the observable placements. Using these market parameters, we can then answer counterfactual questions of how the design of the job market affects the welfare of market participants.

This notebook focuses on the tier classification estimation, which is required to estimate the unobservable types; it does not include estimation of the unobservable types themselves.

## The Data

We use data from the Mapinator, specifically the Mapinator API (https://support.econjobmarket.org/api/mapinator). This API returns a collection of successful placement outcomes for graduates of economics Ph.D. programs. We focus on the submarket for entry-level Ph.D. graduates, so we need to filter the data. The filter process is documented below.

The code for converting API data for use with estimation is adapted from [a notebook](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/2023_summer_projects/api_to_mapinator_julia.ipynb) written by Silas Kwok.

We start by retrieving placements data from the API.

In [2]:
"""
SBM API Data Filter (Julia Version)
Adapted from James Yuming Yu (5 June 2023)

Silas Kwok, 31 July 2023

Adapted and modified for use with full estimation by James Yu, 17 September 2023
"""

using HTTP, JSON
DEBUG_LEVEL = 1

1

In [3]:
function matches(keywords, phrase)
    # checks if any of the keywords are in the phrase
    for keyword in keywords
        if occursin(keyword, phrase)
            return true
        end
    end
    return false
end

matches (generic function with 1 method)

### STEP 1a
Retrieve the placement outcomes.

In [4]:
# NOTE: request times out after 120 seconds. If the data takes longer than 120s to download, adjust the timeout.
placements = nothing
try
    mapinator_data = HTTP.get("https://support.econjobmarket.org/api/mapinator", timeout = 120)
    placements = JSON.parse(String(mapinator_data.body))
catch e
    error("Failed to retrieve data from the API: $e")
end

23973-element Vector{Any}:
 Dict{String, Any}("disappeared" => nothing, "to_shortname" => "Economics, Tsinghua University", "created_at" => "2019-07-02 15:40:46", "to_name" => "Tsinghua University", "to_department" => "Department of Economics, School of Economics and Management", "to_latitude" => 39.999567, "name" => "Development; Growth", "category_id" => 1, "from_shortname" => "Economics, Univ of Michigan", "to_oid" => 623…)
 Dict{String, Any}("disappeared" => nothing, "to_shortname" => "Economics, Amherst College", "created_at" => "2019-07-02 19:38:17", "to_name" => "Amherst College", "to_department" => "Economics", "to_latitude" => 42.3709104, "name" => "Development; Growth", "category_id" => 1, "from_shortname" => "Economics, Univ of Michigan", "to_oid" => 3503…)
 Dict{String, Any}("disappeared" => nothing, "to_shortname" => "Int Food Policy Res Institute", "created_at" => "2019-07-02 19:40:39", "to_name" => "International Food Policy Research Institute (IFPRI)", "to_department" =

### STEP 1b
Group placements by applicant ID and eliminate "oid 893" positions (Ocean and Crow).

In [5]:
# TODO: are the json fields strictly typed? is there a way to easily compensate if the variable types change?

applicant_outcomes = Dict{Any, Vector}()
applicant_ids = Set{Any}()
num_outcomes_selected = 0

for outcome in placements
    push!(applicant_ids, outcome["aid"])
    if outcome["to_oid"] != 893
        push!(get!(applicant_outcomes, outcome["aid"], Vector()), outcome)
        num_outcomes_selected += 1
    end
end

if DEBUG_LEVEL > 0
    println("  ", length(placements), " total placement outcomes")
    println("  -", length(placements) - num_outcomes_selected, " outcomes at Ocean and Crow")
    println("  ", num_outcomes_selected, " outcomes not at Ocean and Crow")
    println()
    println("  ", length(applicant_ids), " total applicants with placements")
    println("  -", length(applicant_ids) - length(applicant_outcomes), " total applicants with exclusively outcomes at Ocean and Crow")
    println("  ", length(applicant_outcomes), " applicants with outcomes not at Ocean and Crow")
end

  23973 total placement outcomes
  -2847 outcomes at Ocean and Crow
  21126 outcomes not at Ocean and Crow

  16854 total applicants with placements
  -1515 total applicants with exclusively outcomes at Ocean and Crow
  15339 applicants with outcomes not at Ocean and Crow


### STEP 2a
Determine the first placement outcome of each individual that occurred after the individual graduated.\
We need to know what the first outcome is BEFORE we filter on types of outcomes, as otherwise we will get incorrectly-identified "first-time positions".

### STEP 2b
Remove postdoc outcomes so applicants with postdoc positions aren't automatically removed from the data.\
Postdocs are concurrent so the placements are redundant on top of e.g. concurrently-awarded assistant professor positions.

In [6]:
postdoc_counter = 0
finalized_applicant_outcomes = Dict{Any, Any}()

for applicant_id in keys(applicant_outcomes)
    for outcome in applicant_outcomes[applicant_id]
        # if you wish to display postdocs in the sinks, remove the if statement condition 
        #   and set Post-Doc to have higher priority than Assistant Professor below
        # alternatively, to only include postdocs in the sinks that did not receive professorships,
        #   do not alter the below code, and instead conduct a second pass 
        #   to fill in postdoc outcomes for individuals with no professorships
        if outcome["position_name"] != "Post-Doc"
            if !haskey(finalized_applicant_outcomes, applicant_id)
                # just add the outcome if the applicant doesn't have any yet
                finalized_applicant_outcomes[applicant_id] = outcome
            else
                # otherwise, the applicant does have at least one other outcome
                if outcome["startdate"] < finalized_applicant_outcomes[applicant_id]["startdate"]
                    # take the earliest outcome of the two and ignore the other
                    finalized_applicant_outcomes[applicant_id] = outcome
                elseif outcome["startdate"] == finalized_applicant_outcomes[applicant_id]["startdate"]
                    # sometimes we may have multiple outcomes that started on the same date - follow priority listing
                    if outcome["position_name"] in ["Assistant Professor"]
                        finalized_applicant_outcomes[applicant_id] = outcome
                    elseif outcome["position_name"] in ["Consultant"] && !(finalized_applicant_outcomes[applicant_id]["position_name"] in ["Assistant Professor"])
                        finalized_applicant_outcomes[applicant_id] = outcome
                    elseif outcome["position_name"] in ["Other Academic", "Other Non-Academic"] && !(finalized_applicant_outcomes[applicant_id]["position_name"] in ["Assistant Professor", "Consultant"])
                        finalized_applicant_outcomes[applicant_id] = outcome
                    end
                end
            end
        else
            postdoc_counter += 1
        end
    end
end

if DEBUG_LEVEL > 0
    println("  -", length(applicant_outcomes) - length(finalized_applicant_outcomes), " total applicants removed due to only being postdocs (", 
        postdoc_counter, " total postdoc placements detected)")
    println("  ", length(finalized_applicant_outcomes), " total applicants ported to finalized collection")
end

  -1759 total applicants removed due to only being postdocs (3352 total postdoc placements detected)
  13580 total applicants ported to finalized collection


### STEP 3
Eliminate everything except:
- Assistant Professor
- Consultant
- Other Academic
- Other Non-Academic

In [7]:
# add "Lecturer" if adjusting sinks later on
valid_labels = Set(["Assistant Professor", "Consultant", "Other Academic", "Other Non-Academic"])
irrelevant_counter = 0
removed_labels = Set()

for applicant_id in copy(keys(finalized_applicant_outcomes))
    outcome = finalized_applicant_outcomes[applicant_id]
    if !(outcome["position_name"] in valid_labels)
        push!(removed_labels, outcome["position_name"])
        delete!(finalized_applicant_outcomes, applicant_id)
        irrelevant_counter += 1
    end
end

if DEBUG_LEVEL > 0
    println("  -", irrelevant_counter, " irrelevant applicants removed from the following classes of positions:")
    println(removed_labels)
    println("  ", length(finalized_applicant_outcomes), " applicants remaining after irrelevant-position applicants removed:")
    maintained_labels = Dict{Any, Int}()
    for applicant_id in keys(finalized_applicant_outcomes)
        outcome = finalized_applicant_outcomes[applicant_id]
        position_name = outcome["position_name"]
        if haskey(maintained_labels, position_name)
            maintained_labels[position_name] += 1
        else
            maintained_labels[position_name] = 1
        end
    end
end

println(maintained_labels, " ", sum(values(maintained_labels)), " total")

  -1797 irrelevant applicants removed from the following classes of positions:
Set(Any["Tenured Professor", "Untenured Professor", "Associate Professor", "Assistant, Associate or Full Professor", "Professor Any Level", "Full Professor", "Temporary Lecturer", "Visiting Professor/Lecturer/Instructor", "Assistant or Associate Professor", "Lecturer"])
  11783 applicants remaining after irrelevant-position applicants removed:
Dict{Any, Int64}("Other Non-Academic" => 2381, "Consultant" => 485, "Other Academic" => 1242, "Assistant Professor" => 7675) 11783 total


### STEP 4
Filter-by-year.

In [8]:
sorted_by_year = Dict{Any, Dict}()
removed_year_placed = 0

remove_years = [] # remove all 2022+ entries
# "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019",
# "2021", "2022", "2023", "2024", "2025", "2026"

for applicant_id in copy(keys(finalized_applicant_outcomes))
    outcome = finalized_applicant_outcomes[applicant_id]
    if matches(remove_years, outcome["startdate"])
        removed_year_placed += 1
        delete!(finalized_applicant_outcomes, applicant_id)
    else
        push!(get!(sorted_by_year, parse(Int, split(outcome["startdate"], "-")[1]), Dict()), applicant_id => outcome)
    end
end

if DEBUG_LEVEL > 0
    println("  -", removed_year_placed, " applicants removed due to placement in 2022/2023/2024")
    println("  ", length(finalized_applicant_outcomes), " applicants remaining after year corrections")
    println()
end

for key in sort(collect(keys(sorted_by_year)))
    println("Year ", key, " has ", length(sorted_by_year[key]), " placement outcomes")
end

  -0 applicants removed due to placement in 2022/2023/2024
  11783 applicants remaining after year corrections

Year 2003 has 29 placement outcomes
Year 2004 has 26 placement outcomes
Year 2005 has 186 placement outcomes
Year 2006 has 27 placement outcomes
Year 2007 has 167 placement outcomes
Year 2008 has 126 placement outcomes
Year 2009 has 262 placement outcomes
Year 2010 has 303 placement outcomes
Year 2011 has 371 placement outcomes
Year 2012 has 424 placement outcomes
Year 2013 has 450 placement outcomes
Year 2014 has 493 placement outcomes
Year 2015 has 727 placement outcomes
Year 2016 has 1438 placement outcomes
Year 2017 has 1005 placement outcomes
Year 2018 has 1048 placement outcomes
Year 2019 has 1252 placement outcomes
Year 2020 has 1218 placement outcomes
Year 2021 has 979 placement outcomes
Year 2022 has 842 placement outcomes
Year 2023 has 360 placement outcomes
Year 2024 has 48 placement outcomes
Year 2025 has 2 placement outcomes


### STEP 5
Save to disk.

In [9]:
total_check = sum(length(value) for value in values(sorted_by_year))
println("Total " * "$total_check" * " applicants in JSON file (compare to" * " $(length(finalized_applicant_outcomes)) " 
    * "applicants in finalized_applicant_outcomes: " * "$(length(finalized_applicant_outcomes) == total_check ? "SUCCESS" : "FAIL")" * ")")

json_str = JSON.json(sorted_by_year, 4)  
open("to_from_by_year_mapinator_api.json", "w") do f
    write(f, json_str)
end

Total 11783 applicants in JSON file (compare to 11783 applicants in finalized_applicant_outcomes: SUCCESS)


13725509

This concludes the API-loading code. We now have the placements, sorted by year:

In [10]:
sorted_by_year

Dict{Any, Dict} with 23 entries:
  2024 => Dict{Any, Any}("57394"=>Dict{String, Any}("disappeared"=>0, "to_short…
  2004 => Dict{Any, Any}("24943"=>Dict{String, Any}("disappeared"=>nothing, "to…
  2023 => Dict{Any, Any}("57593"=>Dict{String, Any}("disappeared"=>0, "to_short…
  2010 => Dict{Any, Any}("5422"=>Dict{String, Any}("disappeared"=>nothing, "to_…
  2006 => Dict{Any, Any}("2139"=>Dict{String, Any}("disappeared"=>nothing, "to_…
  2020 => Dict{Any, Any}("45907"=>Dict{String, Any}("disappeared"=>0, "to_short…
  2003 => Dict{Any, Any}("17827"=>Dict{String, Any}("disappeared"=>nothing, "to…
  2018 => Dict{Any, Any}("37196"=>Dict{String, Any}("disappeared"=>nothing, "to…
  2017 => Dict{Any, Any}("27626"=>Dict{String, Any}("disappeared"=>0, "to_short…
  2025 => Dict{Any, Any}("56703"=>Dict{String, Any}("disappeared"=>0, "to_short…
  2011 => Dict{Any, Any}("9123"=>Dict{String, Any}("disappeared"=>nothing, "to_…
  2009 => Dict{Any, Any}("1886"=>Dict{String, Any}("disappeared"=>nothing, "

We now need to run the setup for the type allocation code on these placements. This is adapted from [create_new_allocation.ipynb](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/create_new_allocation.ipynb).

In [11]:
include("type_allocation_base_by_applicant.jl")
included_years = keys(sorted_by_year)
# auto-detect year interval; change this to select the years of data to include in the estimation
YEAR_INTERVAL = 2013:2015 #minimum(included_years):maximum(included_years)

2013:2015

I focus on a particular subset of the data for computational tractability purposes; increasing the amount of data results in the algorithm below running significantly slower.

In [12]:
to_from_by_year_api = Dict()
for key in keys(sorted_by_year)
    to_from_by_year_api[string(key)] = sorted_by_year[key]
end
# alternatively, load an existing file of placement data
# to_from_by_year_api = SBM_flexible.fetch_data("to_from_by_year.json")

Using the placements, we can sort them into academic and sink placements, as well as collect some labels:

In [13]:
applicant_mapping, department_mapping, applicant_list, department_list, academic, academic_to, academic_builder, rough_sink_builder, department_name_mapping, reverse_mapping, applicant_department_mapping = SBM.get_builders(to_from_by_year_api, YEAR_INTERVAL);

We are mostly flexible in how we choose to design the set of sinks to include. One exception is teaching universities, which are always included:

In [14]:
# sink of teaching universities that do not graduate PhDs
# this must be constructed using academic placements, not pre-defined sink placements

teaching_universities = Set() 
for dept_name in academic_to
    if !(dept_name in academic)
        # the department hired an assistant professor but never graduated anyone
        push!(teaching_universities, dept_name)
    end
end

The rest are built by standardized `if` statements:

In [15]:
public_sector = ("Public Sector", Set())
private_sector = ("Private Sector", Set())
other_groups = ("Other Groups", Set())

postdocs = ("Postdocs", Set())
lecturers = ("Lecturers", Set())
other_academic = ("Other Academic", Set())

for outcome in rough_sink_builder
    if outcome["recruiter_type"] == 5 # government institution
        push!(public_sector[2], (string(outcome["to_name"], " ($(public_sector[1]))"), outcome))
    elseif outcome["recruiter_type"] in [6, 7] # private sector: for and not for profit
        push!(private_sector[2], (string(outcome["to_name"], " ($(private_sector[1]))"), outcome))
    elseif outcome["recruiter_type"] == 8 # international organizations, think tanks, assorted
        push!(other_groups[2], (string(outcome["to_name"], " ($(other_groups[1]))"), outcome))

    # some other examples
    # every example here must also have a corresponding sink Set() above, 
    #     and an entry in sinks_to_include below
   
    elseif outcome["postype"] == 6
        # postdocs that are not in the above (i.e. academic; not public, private, or other)
        # requires reconfiguration of the to_from_by_year loading code
        push!(postdocs[2], (string(outcome["to_name"], " ($(postdocs[1]))"), outcome))
    elseif outcome["postype"] in [5, 7]
        # lecturers that are not in the above
        # requires reconfiguration of the to_from_by_year loading code
        push!(lecturers[2], (string(outcome["to_name"], " ($(lecturers[1]))"), outcome))
    else
        # everything else including terminal academic positions
        # this sink can only be constructed as an "else" statement
        push!(other_academic[2], (string(outcome["to_name"], " ($(other_academic[1]))"), outcome))
    
    end
end

# TODO: if not including a sink results in a department/applicant having zero placements, should it still be included in the adjacency matrix?
# if we know a department exists but has no placements, is this part of the network structure? 
# removing the department means the department does not exist in the network

# sort to ensure consistent ordering
academic_list = sort(collect(academic))
teaching_list = sort(collect(teaching_universities))
# to be consistent with the original estimation, we only include these additional sinks:
sinks_to_include = (public_sector, private_sector, other_groups)#, postdocs, lecturers, other_academic)

sink_mapping, sink_list, sink_builder, sinks, sink_labels = SBM.build_sinks(sinks_to_include, teaching_list)

NUMBER_OF_SINKS = length(sink_labels)
institutions = vcat(academic_list, sinks...)
println("$(length(academic_list)) academic departments, $(length(institutions)) total departments")

Including the following sinks:
 Public Sector
 Private Sector
 Other Groups
 Teaching Universities
Total 4 sinks
359 academic departments, 895 total departments


Next, the adjacency matrix. This matrix represents the individual placements from graduating departments on the columns to hiring employers on the rows:

In [16]:
length(academic_builder) + length(sink_builder)

1520

In [17]:
out = SBM.get_adjacency_by_department(department_mapping, sink_mapping, academic, academic_builder, sink_builder);

Total 1520 Placements (found 1520 by sequence counting, 1520 by matrix sum)


In [18]:
sum(out)

1520

In [19]:
out

895×359 Matrix{Int32}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0   

Total number of applicants:

In [20]:
length(applicant_list)

1670

Note that this does not match the number of placements, likely because several sink placements present in the filtered data are not included (e.g. `other_academic`).

Total number of academic departments:

In [21]:
length(department_list)

359

Total number of sink departments for each sink, in the order they were included (public sector, private sector, other groups, teaching universities):

In [22]:
[length(s) for s in sinks]

4-element Vector{Int64}:
  65
  75
  10
 386

## The Tier Classification Estimation Algorithm

The underlying model of the job market for graduates of economics Ph.D. programs is described in the original Peters & Yu (2023, work in progress) paper. The model conjectures that the types of candidates and employers are discrete, ordered tiers. Under the unique symmetric mixed-strategy equilibrium of the model, the number of graduates from a given department hired by a given employer can be modeled by a Poisson distribution parametrized by a mean depending on the tiers of the department and employer.

This set of Poisson distributions and the tier classification can be estimated with the Stochastic Block Model framework (Karrer & Newman, 2011; Peixoto, 2014; Wang & Bickel, 2017). The algorithm we created as part of the Mapinator project is derived from these approaches, but is different in implementation. Given the number of tiers in the network, the algorithm proceeds as follows:

Initialize the tier of every academic department to 1, and the tier of every sink department to fixed numbers for each sink category.

Repeat the following:
1. Randomly select an academic department.
2. Determine whether the likelihood of the placements being drawn from a Poisson random graph increases when the tier of the department is randomly changed. To compute the likelihood, estimate the Poisson means using the mean placement rates from graduating departments of a given tier to hiring employers of a given tier.
3. If the likelihood increases, keep the change; otherwise, revert the change.
4. Continue until no further likelihood improvements are possible.

We assume initially that the tiers of the candidates are the same as the tiers of the departments they graduated from. I test the algorithm without this assumption later (an idea suggested by Mike Peters).

The algorithm does not produce labels for the tiers. At the moment, we order the tiers based on the number of graduates in each tier such that tier 1 has the greatest number of graduates.

As the number of tiers in the network also needs to be estimated, in general we would also estimate the optimal number of tiers. Previously (as demonstrated in [estimate_model.ipynb](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_model.ipynb)), we used the approach from Wang & Bickel (2017) to do so, which suggested that the optimal number of tiers is four; however, this approach only applies to networks where there is at most one placement between a graduating department and a hiring employer, so an extension of the approach would be required to formally use it here. Moreover, sinks are not present in general frameworks such as those in Karrer & Newman (2011) or Peixoto (2014), which may create additional issues with applying Wang & Bickel (2017). For now, I use this optimal number of tiers to demonstrate the tier classification estimation algorithm:

In [23]:
NUMBER_OF_TYPES = 4 # argmax(candidate_types) if finding optimal number of types
numtotal = NUMBER_OF_TYPES + NUMBER_OF_SINKS

8

Note that since the algorithm differs from previous known algorithms, there is no guarantee that the algorithm will converge to the true tier classification. Based on empirical testing, including bootstrapping with respect to the placements, the algorithm appears to be somewhat robust. Further analysis will allow for a greater understanding of the robustness of the algorithm in the future.

Additionally, my implementation of the algorithm in this notebook differs from past versions in the Mapinator project in that I am classifying individual departments rather than entire institutions; for example, I separately classify several departments at UBC rather than classifying UBC as a whole. This will allow me to compare the results below to previous approaches with classifying entire institutions.

I run the algorithm below. The numerical outputs represent values proportional to the negative log likelihood for each improvement; the exact derivation can be found here: https://github.com/jbrightuniverse/mapinator/tree/master/estimation#readme

The matrices represent the placement rates given intermediate estimations of the tier classification. I print them out after every 100 likelihood improvements.

In [24]:
using Random

### Experiment 1: Cluster by departments, where candidates have the same tiers as their graduating departments

In [25]:
Random.seed!(0) # for reproducibility
num_applicants = 0 # length(applicant_list)
@time est_obj, est_alloc = SBM.doit(out, num_applicants, length(department_list), nothing, [length(s) for s in sinks], NUMBER_OF_TYPES, numtotal, 500 * (NUMBER_OF_TYPES-2) + 1000)

8010.572220711324 



8×4 Matrix{Int32}:
 428  2  0  0
   1  0  0  0
   0  0  0  0
   0  0  0  0
 212  1  0  0
 203  0  0  0
  63  1  0  0
 607  2  0  0


8010.3679463688995 8010.176061507911 8008.0163057386335 8006.0468868306425 8001.06463005163 8000.476314878059 7999.628050635904 7999.520034781973 7996.096712827148 7992.068835364156 7990.048237376206 7985.854372050717 7981.086211514474 7976.760688573823 7974.081727747662 7969.166517421932 7966.3147846488055 7953.300603855871 7950.745046124186 7950.341023889183 7946.086223578779 7935.638133556638 7908.48624341567 7891.860133249406 7887.677879050806 7886.064145280181 7861.929014767766 7860.796582589389 7858.865389855545 7854.9953499116045 7852.8019050625535 7848.6019102840555 7844.330572931089 7833.514245513005 7805.555720684822 7801.3386216104955 7799.4535280760665 7795.191533355526 7758.916771491123 7710.934033439209 7651.908725575407 7647.984782929756 7645.999859828017 7644.13299279315 7549.169999090258 7547.234449265151 7545.5019879898355 7537.583729200583 7533.854407711369 7451.205215931701 7451.189802867702 7445.5912475413625 7429.740006237982 7429.45521562232 7414.957114716771 74

8×4 Matrix{Int32}:
 121   63   0   81
  38   18   0   32
   6    5   3    4
  26   18   0   16
  89   61   5   58
  98   49   5   51
  22   15   0   27
 282  178  26  123


6867.731544482341 6863.726186260312 6862.103165096429 6861.175014146911 6859.239421875098 6857.302134163314 6855.584174337424 6822.096522003454 6819.756373624186 6817.386750971903 6814.988097850211 6813.248913923608 6812.762767880589 6810.337585609232 6807.884227349356 6806.558744527325 6804.068791645172 6802.588260528359 6800.685004624823 6798.616747646851 6796.10873983861 6795.1810964114975 6794.043753537071 6790.87665096489 6789.1569780745085 6787.595076195711 6780.319000212519 6778.640537976594 6778.201520870037 6776.480116852752 6774.711102565393 6773.7570876747695 6773.077076828563 6771.21741734635 6769.412326877041 6767.466377226871 6767.098205957236 6765.4441840717745 6764.457165872517 6762.615759280966 6748.9009801335615 6747.789451731043 6743.732486842813 6742.098644111038 6740.438025351011 6738.751151895849 6737.038507469569 6735.688634302356 6733.8876436806495 6724.019560809589 6675.67630336001 6674.075742662599 6660.349155496555 6643.048479356624 6620.110207490619 6605.20

8×4 Matrix{Int32}:
  57   53   0  109
  18   10   0   72
   0    4  11    6
  10   13   0   68
  53   33  15  112
  55   38   9  101
  19   11   0   34
 162  157  53  237


6273.475723549603 6271.3701340280095 6269.2411208066615 6267.088653716791 6255.342964825678 6253.810139546866 6253.695087300539 6252.706088427879 6230.105798594438 6228.883783666812 6227.36820051189 6225.256513339248 6224.980189597058 6223.847418548652 6222.45751011582 6221.462466602788 6219.920959377989 6218.357544976633 6217.78080483529 6214.518580838584 6212.4313682455195 6210.887171793794 6209.320929063267 6207.732716178598 6206.712755747384 6204.557761677933 6203.185024509402 6201.0072324233915 6198.804750784689 6197.176614668439 6195.525823409385 6193.85241264914 6192.156407495757 6191.403228833011 6190.029176929973 6189.990519424179 6188.298328412318 6186.583359709949 6183.169427001105 6168.315813721662 6168.270547414824 6165.943308191011 6164.173581829093 6159.47027628535 6157.104588062607 6154.71295610236 6151.953308697217 6151.689922476204 6149.337077326487 6148.311829487095 6146.485459564323 6144.0682870895 6139.264530503483 6136.352328548786 6134.643543803809 6130.44160716

8×4 Matrix{Int32}:
 31   81   0   80
 20   30   0  103
  0    0  17    0
  4   13   0   52
 24   63  28   98
 33   61  18   91
 12   19   0   33
 95  227  83  204


  4.883040 seconds (555.93 k allocations: 37.248 MiB, 0.31% gc time, 8.76% compilation time)


(6036.890073316794, Int32[3, 3, 1, 2, 3, 1, 2, 3, 2, 3  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [26]:
placement_rates, counts, sorted_allocation, full_likelihood = SBM.get_allocation(est_alloc, out, num_applicants, NUMBER_OF_TYPES, numtotal)

(Int32[52 13 4 0; 103 30 20 0; … ; 33 19 12 0; 204 227 95 83], Int32[256 672 1984 2832; 672 1764 5208 7434; … ; 160 420 1240 1770; 6176 16212 47864 68322], Int32[4, 4, 3, 2, 4, 3, 2, 4, 2, 4  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8], -7678.7174635781375)

We can explore the results:

In [27]:
placement_rates

8×4 Matrix{Int32}:
  52   13   4   0
 103   30  20   0
  80   81  31   0
   0    0   0  17
  98   63  24  28
  91   61  33  18
  33   19  12   0
 204  227  95  83

In [28]:
counts

8×4 Matrix{Int32}:
  256    672   1984   2832
  672   1764   5208   7434
 1984   5208  15376  21948
 2832   7434  21948  31329
 1040   2730   8060  11505
 1200   3150   9300  13275
  160    420   1240   1770
 6176  16212  47864  68322

In [29]:
placement_rates ./ counts # means

8×4 Matrix{Float64}:
 0.203125   0.0193452  0.00201613  0.0
 0.153274   0.0170068  0.00384025  0.0
 0.0403226  0.015553   0.00201613  0.0
 0.0        0.0        0.0         0.000542628
 0.0942308  0.0230769  0.00297767  0.00243372
 0.0758333  0.0193651  0.00354839  0.00135593
 0.20625    0.0452381  0.00967742  0.0
 0.0330311  0.014002   0.00198479  0.00121484

In [30]:
full_likelihood

-7678.7174635781375

In [31]:
SBM.nice_table(placement_rates, NUMBER_OF_TYPES, NUMBER_OF_SINKS, sink_labels)

┌───────────────────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │     52 │     13 │      4 │      0 │         69 │
│                Tier 2 │    103 │     30 │     20 │      0 │        153 │
│                Tier 3 │     80 │     81 │     31 │      0 │        192 │
│                Tier 4 │      0 │      0 │      0 │     17 │         17 │
│         Public Sector │     98 │     63 │     24 │     28 │        213 │
│        Private Sector │     91 │     61 │     33 │     18 │        203 │
│          Other Groups │     33 │     19 │     12 │      0 │         64 │
│ Teaching Universities │    204 │    227 │     95 │     83 │        609 │
│         Column Totals │    661 │    494 │    219 │    146 │       1520 │
└───────────────────────┴────────┴────────┴────────┴────────┴────────────┘
\begin{tabular}{rrrrrr}
 

In [32]:
for sorted_type in 1:NUMBER_OF_TYPES+NUMBER_OF_SINKS
    counter = 0
    inst_hold = []
    println("TYPE $sorted_type:")
    for (i, sbm_type) in enumerate(sorted_allocation)
        if sbm_type == sorted_type
            if sorted_type <= NUMBER_OF_TYPES
                push!(inst_hold, department_name_mapping[department_list[i]])
            else
                push!(inst_hold, sink_list[i - length(department_list)])
            end
            counter += 1
        end
    end
    for inst in sort(inst_hold)
        println("  ", inst)
    end
    println("Total Departments: $counter")
    println()
end

TYPE 1:
  Boston University - Economics
  Columbia University - Economics
  Duke University - Economics
  Harvard University - Department of Economics
  New York University - Economics
  Northwestern University - Economics
  Princeton University - Economics
  Stanford University - Department of Economics
  University of California Los Angeles (UCLA) - Economics
  University of California, Berkeley - Economics
  University of Chicago - Economics
  University of Maryland - Economics
  University of Michigan - Department of Economics
  University of Pennsylvania - Department of Economics
  University of Wisconsin, Madison - Economics
  Yale University - Economics
Total Departments: 16

TYPE 2:
  Bocconi University - Economics
  Boston College - Department of Economics
  Brown University - Economics
  Carnegie Mellon University - Tepper School of Business
  Cornell University - Economics
  European University Institute - Economics Department
  Johns Hopkins University - Economics
  Kansas 

To save the allocation to file, if we want to explore it later, we can do the following:

In [33]:
type_dictionary = []
for (i, alloc) in enumerate(sorted_allocation)
    if alloc in 1:NUMBER_OF_TYPES
        oid = department_list[i]
        push!(type_dictionary, Dict("name" => department_name_mapping[oid], "oid" => oid, "type" => alloc))
    end
end

In [34]:
mkpath(".estimates")
open(".estimates/id_to_type_api.json", "w") do f
    write(f, JSON.json(type_dictionary))
end;

There is some consistency between the results here and previous Mapinator project results such as those in [estimate_model.ipynb](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_model.ipynb); there are some differences in the structure of the adjacency matrix, which could either be due to the individual classification of departments or a result of the smaller amount of data used for the estimation compared to previous estimations.

An interesting observation is that the highest tier is entirely composed of economics departments. Several other departments of these universities have lower tiers, despite being from the same university.

One underlying assumption, as mentioned earlier, is that the tiers of the candidates are the same as the tiers of their graduating departments. If I relax this assumption so that candidates and departments have separate tier classifications, we can see if the resulting estimated classifications would change. This idea was suggested by Mike Peters.

I initially classified both departments and candidates at the same time; however, the resulting classification simply created candidate tiers where each tier would map directly to a different sink tier or academic tier. I instead tried taking the department classification from above as given, classifying only the candidates below, but this produced a similar result as the classification with both candidates and departments at the same time. This estimation is below.

Unlike the department classification, where each graduating department had its own column in the placement matrix, this classification requires a placement matrix where each individual applicant has their own column:

In [35]:
out2 = SBM.get_adjacency(applicant_mapping, department_mapping, sink_mapping, academic, academic_builder, sink_builder);

Total 1520 Placements (found 1520 by sequence counting, 1520 by matrix sum)


In [36]:
sum(out2)

1520

In [37]:
out2

895×1670 Matrix{Int32}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

This is a limitation in terms of how much data can be used as the matrix increases in the number of applicants; the greater the number of placements used in the estimation, the bigger the matrix, and the slower the algorithm.

One property of this placements matrix not present in the previous placements matrix is that every entry is either zero or one as every applicant has at most one placement in their column, making the matrix compatible with Wang & Bickel (2017). However, because the departments are classified using the original placements matrix with entries greater than one, the full approach here is still not entirely compatible with the approach in Wang & Bickel (2017). Moreover, the departments are classified separately from candidates and the sinks are still present; these do not match general frameworks such as those in Karrer & Newman (2011) or Peixoto (2014), which may again create additional issues when trying to apply Wang & Bickel (2017).

### Experiment 2: Cluster by candidates, taking the department classification as given, with four candidate tiers

In [38]:
Random.seed!(0) # for reproducibility
num_applicants2 = length(applicant_list)
@time est_obj2, est_alloc2 = SBM.doit(out2, num_applicants2, length(department_list), sorted_allocation[1:length(department_list)], [length(s) for s in sinks], NUMBER_OF_TYPES, numtotal, 500 * (NUMBER_OF_TYPES-2) + 1000)

8×4 Matrix{Int32}:
  69  0  0  0
 152  1  0  0
 192  0  0  0
  17  0  0  0
 213  0  0  0
 203  0  0  0
  64  0  0  0
 609  0  0  0

10082.786349339422 


10082.097751201216 10080.079364334037 10079.169073277619 10077.613554451707 10077.389934764165 10076.58996096778 10076.53095413437 10075.982186742785 10075.404915289864 10074.953807175754 10074.736493068556 10074.207867212 10072.298346237778 10072.296486443494 10071.8987806063 10071.422486089177 10070.887963019848 10070.304405251545 10070.256818299571 10069.419524820996 10069.014693876734 10067.346893142294 10066.460547972212 10066.45311792986 10066.076938779677 10065.23725130034 10065.183211954673 10063.415444484104 10063.001789259444 10061.173166771596 10060.922796224899 10060.189054753611 10058.82535493409 10058.312520221984 10058.130158606635 10057.586314089483 10056.173346374484 10055.603259304746 10054.031121302973 10053.438138831998 10053.232900840574 10053.01107486252 10052.397987231374 10052.152769896864 10050.664152949208 10050.145938154223 10049.562572200139 10048.084119861805 10047.62016665665 10046.72775174698 10045.603900173232 10044.54284708102 1004

8×4 Matrix{Int32}:
  58   5   2   4
 143  10   0   0
 183   9   0   0
  16   0   0   1
 212   0   0   1
 185   0   8  10
  61   3   0   0
 584   0  25   0


10007.310434206289 10007.13843294811 10006.570459898823 10004.716721227305 10004.1717515034 10002.79874715948 10001.822836821697 10001.244807822894 10000.655353926943 10000.055112041848 9999.544997916622 9998.959934581992 9998.364602662303 9996.333157264411 9996.298672503925 9996.120132302882 9995.879548235962 9995.277695416513 9994.426460152023 9993.815919339324 9993.625016452386 9993.358691776064 9991.21874775051 9990.300310616603 9989.414534135643 9988.124059095691 9987.84685478037 9987.230312246284 9986.754037873354 9984.64638714713 9984.042756033958 9983.87870184785 9983.672688341285 9983.645525689328 9983.03338304196 9981.70064548935 9981.080954350198 9980.193956199908 9979.567055580635 9977.285576535449 9976.651781867837 9976.352113101686 9974.996796807973 9973.593214126611 9972.953573956576 9971.976807452618 9969.612940551575 9968.19060109471 9967.044821356707 9966.400974630544 9963.952339383719 9963.542608194139 9963.358593307825 9963.124617855938 9962.274434437157 9960.57274

8×4 Matrix{Int32}:
  46   8   3  12
 133  20   0   0
 174  18   0   0
  15   0   0   2
 213   0   0   0
 176   0  12  15
  58   6   0   0
 557   0  52   0

...

8×4 Matrix{Int32}:
   1    6    0   62
   8  145    0    0
  17  175    0    0
   3    0    0   14
 213    0    0    0
   0    0   18  185
   5   59    0    0
  47    0  562    0


8402.018824871504 8398.93615353214 8397.123460946625 8395.292388276017 8393.208155343871 8390.464973559268 8388.315954297317 8385.174082413283 8383.32932153996 8381.465242746845 8379.58132953565 8377.376395836984 8377.076661202738 8374.416720563506 8374.385930806804 8372.488664861838 8369.277955492293 8367.009786761993 8366.709051340167 8364.792686791337 8364.487899919503 8362.550033687754 8360.296724528109 8358.340299548547 8356.361095103732 8354.358412750113 8352.039442023917 8349.538070008613 8346.253524787408 8344.232185423538 8342.185878857366 8340.113777910006 8339.805549670173 8336.444995111298 8334.344526811747 8331.60102443611 8328.689518271949 8326.568867783913 8324.419646878392 8322.24079381233 8319.138664920663 8318.82702921534 8315.471750891047 8311.77631902162 8311.436662053107 8311.119255334876 8307.677197610827 8305.477260995014 8301.94982960014 8299.716266734327 8296.757322004283 8294.42400547332 8291.64126652827 8289.382164884544 8287.088436428583 8286.769483617307 8

8×4 Matrix{Int32}:
   0    0    0   69
   2  151    0    0
   6  186    0    0
   1    0    0   16
 213    0    0    0
   0    0    2  201
   0   64    0    0
  10    0  599    0


8158.020588393663 8154.723152202662 8150.648195430195 8147.689319572602 8147.355230841043 8143.945562203644 8140.791240963536 8137.247049857462 8133.539850753403 8129.635755350252 8129.297682120841 8125.145394052325 8121.757870583995 8117.274808435764 8111.861417914642 8111.519364129711 8107.805317305415 8102.81270047124 8097.215713035041 8093.7731352794035 8093.372179161267 8092.739728138005 8086.792683799375 continue 
8082.56640778351 8076.198098950739 continue 
continue 
continue 
continue 
8070.592068024351 131.131421 seconds (167.87 k allocations: 11.710 MiB, 0.06% compilation time)


(8070.592068024351, Int32[3, 3, 2, 2, 3, 1, 1, 1, 4, 2  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [39]:
placement_rates2, counts2, sorted_allocation2, full_likelihood2 = SBM.get_allocation(est_alloc2, out2, num_applicants2, NUMBER_OF_TYPES, numtotal, sort_tiers = false)

(Int32[0 0 0 69; 0 153 0 0; … ; 0 64 0 0; 0 0 609 0], Int32[3408 6544 9744 7024; 8946 17178 25578 18438; … ; 2130 4090 6090 4390; 82218 157874 235074 169454], Int32[3, 3, 2, 2, 3, 1, 1, 1, 4, 2  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8], -9590.592068067146)

In [40]:
placement_rates2

8×4 Matrix{Int32}:
   0    0    0   69
   0  153    0    0
   0  192    0    0
   0    0    0   17
 213    0    0    0
   0    0    0  203
   0   64    0    0
   0    0  609    0

In [41]:
counts2

8×4 Matrix{Int32}:
  3408    6544    9744    7024
  8946   17178   25578   18438
 26412   50716   75516   54436
 37701   72393  107793   77703
 13845   26585   39585   28535
 15975   30675   45675   32925
  2130    4090    6090    4390
 82218  157874  235074  169454

In [42]:
placement_rates2 ./ counts2 # means

8×4 Matrix{Float64}:
 0.0        0.0         0.0         0.00982346
 0.0        0.00890674  0.0         0.0
 0.0        0.00378579  0.0         0.0
 0.0        0.0         0.0         0.000218782
 0.0153846  0.0         0.0         0.0
 0.0        0.0         0.0         0.00616553
 0.0        0.0156479   0.0         0.0
 0.0        0.0         0.00259067  0.0

In [43]:
full_likelihood2

-9590.592068067146

In [44]:
SBM.nice_table(placement_rates2, NUMBER_OF_TYPES, NUMBER_OF_SINKS, sink_labels)

┌───────────────────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │      0 │      0 │      0 │     69 │         69 │
│                Tier 2 │      0 │    153 │      0 │      0 │        153 │
│                Tier 3 │      0 │    192 │      0 │      0 │        192 │
│                Tier 4 │      0 │      0 │      0 │     17 │         17 │
│         Public Sector │    213 │      0 │      0 │      0 │        213 │
│        Private Sector │      0 │      0 │      0 │    203 │        203 │
│          Other Groups │      0 │     64 │      0 │      0 │         64 │
│ Teaching Universities │      0 │      0 │    609 │      0 │        609 │
│         Column Totals │    213 │    409 │    609 │    289 │       1520 │
└───────────────────────┴────────┴────────┴────────┴────────┴────────────┘
\begin{tabular}{rrrrrr}
 

With the classification of departments taken as given, the candidates are classified into tiers based on where they were hired. This seems intuitive; unlike before, where applicants had common graduating departments and hiring employers, here applicants only have common hiring employers as applicants appear individually in the placements matrix. As a result, they can only cluster based on this information, which may result in the above matrix.

Several applicant clusters map to multiple academic tiers or sinks. Note that a classification where every hiring tier has a unique cluster of candidates is impossible as there are not enough candidate tiers. I ran some tests to see what would happen if additional candidate tiers were added: with a total of eight tiers, the minimum amount required for exact matching between hiring tiers and candidate tiers, exact matching does not occur. This also does not occur with nine tiers. With 15 tiers, there is exact matching with some redundant tiers, and with 10 tiers, there is exact matching with fewer redundant tiers. In some of these tests, several hiring tiers mapped to multiple candidate tiers; this could either be due to the algorithm identifying differences between the groups of candidates based on the exact hiring employers, or due to the algorithm reaching a local maximum during maximization of the likelihood. I provide the estimation with 10 candidate tiers later in this notebook.

Below is the candidate classification for the estimation with four candidate tiers. Each tier includes the departments whose graduates are in the tier, the number of graduates from the department who are in the tier, as well as the tier of the department from the previous department classification.

In [45]:
for sorted_type in 1:NUMBER_OF_TYPES
    counter = 0
    inst_hold = Dict()
    println("TYPE $sorted_type:")
    for (i, sbm_type) in enumerate(sorted_allocation2[1:length(applicant_list)])
        if sbm_type == sorted_type
            applicant_id = applicant_list[i]
            oid = applicant_department_mapping[applicant_id]
            department_name = department_name_mapping[oid]
            if !((department_name, oid) in keys(inst_hold))
                inst_hold[(department_name, oid)] = 0
            end
            inst_hold[(department_name, oid)] += 1
            counter += 1
        end
    end

    departments = []
    for ((department_name, oid), num_applicants) in inst_hold
        push!(departments, (-num_applicants, department_name, oid))
    end

    for (num_applicants_rev, department_name, oid) in sort(departments)
        num_applicants = -num_applicants_rev
        if num_applicants == 1
            applicant_count_text = "applicant"
        else
            applicant_count_text = "applicants"
        end
        println("  ", department_name, " (Type ", sorted_allocation[department_mapping[oid]], ", ", num_applicants, " ", applicant_count_text, ")")
    end
    println("Total Departments: $counter")
    println()
end

TYPE 1:
  University of Michigan - Department of Economics (Type 1, 17 applicants)
  University of Maryland - Economics (Type 1, 10 applicants)
  University of Wisconsin, Madison - Economics (Type 1, 10 applicants)
  University of Pennsylvania - Department of Economics (Type 1, 8 applicants)
  Boston University - Economics (Type 1, 7 applicants)
  Columbia University - Economics (Type 1, 7 applicants)
  Duke University - Economics (Type 1, 7 applicants)
  Harvard University - Department of Economics (Type 1, 7 applicants)
  University of Virginia - Economics (Type 2, 7 applicants)
  Yale University - Economics (Type 1, 7 applicants)
  Princeton University - Economics (Type 1, 5 applicants)
  Pennsylvania State University - Economics (Type 2, 4 applicants)
  Stockholm School of Economics - Department of Economics (Type 2, 4 applicants)
  University of California Los Angeles (UCLA) - Economics (Type 1, 4 applicants)
  European University Institute - Economics Department (Type 2, 3 applic

### Experiment 3: Cluster by candidates, taking the department classification as given, with 10 candidate tiers

In [46]:
Random.seed!(0) # for reproducibility
NUMBER_OF_TYPES3 = 10
numtotal3 = NUMBER_OF_TYPES3 + NUMBER_OF_SINKS
@time est_obj3, est_alloc3 = SBM.doit(out2, num_applicants2, length(department_list), sorted_allocation[1:length(department_list)], [length(s) for s in sinks], NUMBER_OF_TYPES3, numtotal3, 500 * (NUMBER_OF_TYPES3-2) + 1000)

10082.786349339422 



14×10 Matrix{Int32}:
  69  0  0  0  0  0  0  0  0  0
 152  1  0  0  0  0  0  0  0  0
 192  0  0  0  0  0  0  0  0  0
  17  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0
 213  0  0  0  0  0  0  0  0  0
 203  0  0  0  0  0  0  0  0  0
  64  0  0  0  0  0  0  0  0  0
 609  0  0  0  0  0  0  0  0  0


10082.097751201216 10081.179622263495 10079.16188925479 10078.24342372133 10077.33324045574 10076.413836176496 10075.723327889427 10074.803584270587 10073.882843812053 10073.773689138936 10071.75067372575 10071.172737867651 10070.252970276955 10070.136470652458 10067.735246008015 10066.825055475285 10066.598806923495 10066.07384863717 10064.632157111726 10063.71308021821 10063.185052673287 10062.603650579345 10061.430838705668 10061.321027792472 10060.667686754956 10060.274165768684 10059.093774860457 10059.04667788792 10057.59127439341 10056.40193054423 10056.254539475556 10055.85887022472 10055.313802917006 10055.072808596085 10053.954566953313 10053.311596902118 10052.971280755777 10052.36271850238 10051.66135016551 10050.46761698571 10048.998712911509 10047.455965675306 10046.910270576183 10045.634120780956 10045.015091800018 10044.561065125295 10042.765229264105 10042.02847366835 10041.385149061336 10040.223302245306 10038.810328830406 10038.11028294241 10037.675750192959 10036.7

14×10 Matrix{Int32}:
  61  0  3   0  1  0  0  2  1  1
 150  3  0   0  0  0  0  0  0  0
 182  3  2   0  5  0  0  0  0  0
  16  0  0   0  0  0  0  0  0  1
   0  0  0   0  0  0  0  0  0  0
   0  0  0   0  0  0  0  0  0  0
   0  0  0   0  0  0  0  0  0  0
   0  0  0   0  0  0  0  0  0  0
   0  0  0   0  0  0  0  0  0  0
   0  0  0   0  0  0  0  0  0  0
 206  0  0   0  0  0  6  1  0  0
 174  5  7   2  1  0  5  4  1  4
  58  0  2   0  0  1  2  0  0  1
 580  0  1  13  0  5  0  0  8  2


9994.177167291584 9993.406406439663 9992.340324638197 9991.501492757796 9991.371379822358 9990.780169883084 9990.158302893706 9988.867513969304 9987.662024654788 9986.755381395746 9985.97474996932 9985.066024353753 9984.27579183355 9983.571084067797 9983.400742477676 9983.15946526304 9982.268869724778 9981.472430528805 9980.668964868439 9980.65386005704 9978.60501315377 9977.364053003923 9976.71471362365 9976.043654942367 9974.770729750531 9973.44593366333 9972.535304153169 9972.352999841827 9971.442019795246 9970.639688256068 9970.01240921273 9969.365273203268 9968.557648479818 9968.511826854132 9968.246569543604 9967.348584930049 9966.136156030934 9965.306940137934 9964.394606217134 9963.63192575766 9962.71923574959 9961.060908127993 9960.413118185776 9959.498998618683 9958.689602408993 9957.710631641236 9956.895771833024 9956.075189088322 9955.91585281285 9955.247644366482 9953.064750911728 9952.581506859517 9950.32103461346 9949.496911476235 9948.244384439115 9947.054155981808 994

...

14×10 Matrix{Int32}:
  0    0   1  0   0    0  4    0  64    0
  0  153   0  0   0    0  0    0   0    0
 94    0   0  0  98    0  0    0   0    0
  0    0  17  0   0    0  0    0   0    0
  0    0   0  0   0    0  0    0   0    0
  0    0   0  0   0    0  0    0   0    0
  0    0   0  0   0    0  0    0   0    0
  0    0   0  0   0    0  0    0   0    0
  0    0   0  0   0    0  0    0   0    0
  0    0   0  0   0    0  0    0   0    0
  0    0   0  1   0    0  0  212   0    0
  0    0   0  0   0    0  1    0   1  201
  0    0  64  0   0    0  0    0   0    0
  0    0   0  3   0  606  0    0   0    0


7464.325204707307 7463.995902037817 7463.66272626892 7463.325604613403 7462.984462336641 7462.639222687125 7462.289806823839 7461.936133740277 7461.578120184966 7461.215680578231 7457.366098262111 7457.0263001885205 7456.65934653532 7456.287788333913 7455.911532480089 7455.530483166721 7455.144541778348 7454.753606780481 7454.35757360331 7453.956334519457 7453.549778515403 7453.13779115617 7452.720254442846 7452.297046662445 7451.86804222962 7451.344794085856 7450.909863375912 7450.468872548504 7450.021683361396 7449.568152973575 7449.108133736958 7448.641472975875 7448.168012753431 7447.687589623764 7446.301295262645 7445.8137400080395 7445.31887735971 7444.816525701008 7444.306496751655 7443.788595231497 7443.262618502096 7442.728356184351 7442.517261091389 7441.974494657152 7441.422996992651 7440.298660921162 7439.738195859734 7439.168517681226 7437.80651572772 7437.227368621498 7435.719409265221 7435.130526753512 7434.531631071346 7433.922432499023 7433.3026286424665 7431.67917462

14×10 Matrix{Int32}:
   0    0   0   0   0    0  2    0  67    0
   0  153   0   0   0    0  0    0   0    0
 169    0   0   0  23    0  0    0   0    0
   0    0   1  14   0    0  2    0   0    0
   0    0   0   0   0    0  0    0   0    0
   0    0   0   0   0    0  0    0   0    0
   0    0   0   0   0    0  0    0   0    0
   0    0   0   0   0    0  0    0   0    0
   0    0   0   0   0    0  0    0   0    0
   0    0   0   0   0    0  0    0   0    0
   0    0   0   0   0    0  0  213   0    0
   0    0   0   0   0    0  2    0   0  201
   0    0  64   0   0    0  0    0   0    0
   0    0   0   0   0  609  0    0   0    0


7373.725932429385 7372.523951407735 7371.206878515657 7369.968238291218 7368.69065000067 7367.371575008421 7366.008182983422 7364.597300079162 7363.481582322591 7362.019626356698 7360.502524639647 7356.451587188021 7355.33586943145 7353.086528852975 7351.509612441538 7349.867477703382 7348.153801492097 7346.361088700447 7344.480318102005 7342.500429408827 7340.4075510721 7338.183780706178 7338.0544780789505 continue 
7335.805137500475 7333.2378344432245 7330.430678376222 7327.296668158622 7326.773420014857 7323.129233171099 continue 
continue 
7320.02149583917 continue 
7318.111953334284 continue 
7316.725658973165 281.897299 seconds (318.25 k allocations: 19.743 MiB, 0.01% compilation time)


(7316.725658973165, Int32[6, 6, 2, 2, 6, 8, 8, 8, 5, 1  …  14, 14, 14, 14, 14, 14, 14, 14, 14, 14])

In [47]:
placement_rates3, counts3, sorted_allocation3, full_likelihood3 = SBM.get_allocation(est_alloc3, out2, num_applicants2, NUMBER_OF_TYPES3, numtotal3, sort_tiers = 2)

(Int32[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 609 0 … 0 0], Int32[75516 26412 … 124 18600; 25578 8946 … 42 6300; … ; 6090 2130 … 10 1500; 235074 82218 … 386 57900], Int32[1, 1, 5, 5, 1, 2, 2, 2, 10, 4  …  14, 14, 14, 14, 14, 14, 14, 14, 14, 14], -8836.725659006537)

In [48]:
placement_rates3

14×10 Matrix{Int32}:
   0    0    0  192    0   0   0   0  0  0
   0    0    0    0  153   0   0   0  0  0
   0    0    0    0    0  69   0   0  0  0
   0    0    0    0    0   0   0  17  0  0
   0    0    0    0    0   0   0   0  0  0
   0    0    0    0    0   0   0   0  0  0
   0    0    0    0    0   0   0   0  0  0
   0    0    0    0    0   0   0   0  0  0
   0    0    0    0    0   0   0   0  0  0
   0    0    0    0    0   0   0   0  0  0
   0  213    0    0    0   0   0   0  0  0
   0    0  202    0    0   0   0   0  1  0
   0    0    0    0    0   0  64   0  0  0
 609    0    0    0    0   0   0   0  0  0

In [49]:
counts3

14×10 Matrix{Int32}:
  75516  26412  25048  23808  18972   8556   7936  2108  124  18600
  25578   8946   8484   8064   6426   2898   2688   714   42   6300
   9744   3408   3232   3072   2448   1104   1024   272   16   2400
 107793  37701  35754  33984  27081  12213  11328  3009  177  26550
      0      0      0      0      0      0      0     0    0      0
      0      0      0      0      0      0      0     0    0      0
      0      0      0      0      0      0      0     0    0      0
      0      0      0      0      0      0      0     0    0      0
      0      0      0      0      0      0      0     0    0      0
      0      0      0      0      0      0      0     0    0      0
  39585  13845  13130  12480   9945   4485   4160  1105   65   9750
  45675  15975  15150  14400  11475   5175   4800  1275   75  11250
   6090   2130   2020   1920   1530    690    640   170   10   1500
 235074  82218  77972  74112  59058  26634  24704  6562  386  57900

In [50]:
placement_rates3 ./ counts3 # means

14×10 Matrix{Float64}:
   0.0           0.0          0.0        …    0.0           0.0          0.0
   0.0           0.0          0.0             0.0           0.0          0.0
   0.0           0.0          0.0             0.0           0.0          0.0
   0.0           0.0          0.0             0.00564972    0.0          0.0
 NaN           NaN          NaN             NaN           NaN          NaN
 NaN           NaN          NaN          …  NaN           NaN          NaN
 NaN           NaN          NaN             NaN           NaN          NaN
 NaN           NaN          NaN             NaN           NaN          NaN
 NaN           NaN          NaN             NaN           NaN          NaN
 NaN           NaN          NaN             NaN           NaN          NaN
   0.0           0.0153846    0.0        …    0.0           0.0          0.0
   0.0           0.0          0.0133333       0.0           0.0133333    0.0
   0.0           0.0          0.0             0.0           0.0  

In [51]:
full_likelihood3

-8836.725659006537

In [52]:
SBM.nice_table(placement_rates3, NUMBER_OF_TYPES3, NUMBER_OF_SINKS, sink_labels)

┌───────────────────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬─────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Tier 5 │ Tier 6 │ Tier 7 │ Tier 8 │ Tier 9 │ Tier 10 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┼────────────┤
│                Tier 1 │      0 │      0 │      0 │    192 │      0 │      0 │      0 │      0 │      0 │       0 │        192 │
│                Tier 2 │      0 │      0 │      0 │      0 │    153 │      0 │      0 │      0 │      0 │       0 │        153 │
│                Tier 3 │      0 │      0 │      0 │      0 │      0 │     69 │      0 │      0 │      0 │       0 │         69 │
│                Tier 4 │      0 │      0 │      0 │      0 │      0 │      0 │      0 │     17 │      0 │       0 │         17 │
│                Tier 5 │      0 │      0 │      0 │      0 │      0 │      0 │      0 │  

This produces an exact mapping between hiring tiers and candidate tiers, with one redundant candidate tier and one candidate tier that has only one applicant in it. This may be due to a local optimality issue; below, the graduating department of the applicant is also present in the other candidate tier corresponding to the same hiring tier:

In [53]:
for sorted_type in 1:NUMBER_OF_TYPES3
    counter = 0
    inst_hold = Dict()
    println("TYPE $sorted_type:")
    for (i, sbm_type) in enumerate(sorted_allocation3[1:length(applicant_list)])
        if sbm_type == sorted_type
            applicant_id = applicant_list[i]
            oid = applicant_department_mapping[applicant_id]
            department_name = department_name_mapping[oid]
            if !((department_name, oid) in keys(inst_hold))
                inst_hold[(department_name, oid)] = 0
            end
            inst_hold[(department_name, oid)] += 1
            counter += 1
        end
    end

    departments = []
    for ((department_name, oid), num_applicants) in inst_hold
        push!(departments, (-num_applicants, department_name, oid))
    end

    for (num_applicants_rev, department_name, oid) in sort(departments)
        num_applicants = -num_applicants_rev
        if num_applicants == 1
            applicant_count_text = "applicant"
        else
            applicant_count_text = "applicants"
        end
        println("  ", department_name, " (Type ", sorted_allocation[department_mapping[oid]], ", ", num_applicants, " ", applicant_count_text, ")")
    end
    println("Total Departments: $counter")
    println()
end

TYPE 1:
  University of Michigan - Department of Economics (Type 1, 26 applicants)
  Boston University - Economics (Type 1, 23 applicants)
  University of Wisconsin, Madison - Economics (Type 1, 20 applicants)
  University of Washington - Department of Economics (Type 2, 17 applicants)
  University of Maryland - Economics (Type 1, 16 applicants)
  University of California, Berkeley - Economics (Type 1, 14 applicants)
  Yale University - Economics (Type 1, 14 applicants)
  Harvard University - Department of Economics (Type 1, 13 applicants)
  Purdue University - Economics, Krannert School of Management (Type 2, 13 applicants)
  University of California Los Angeles (UCLA) - Economics (Type 1, 13 applicants)
  Duke University - Economics (Type 1, 11 applicants)
  London School of Economics and Political Science - Department of Economics (Type 2, 11 applicants)
  University of Pennsylvania - Department of Economics (Type 1, 11 applicants)
  Boston College - Department of Economics (Type 2,

As mentioned earlier, the exact mapping between hiring tiers and candidate tiers may be due to the fact that applicants only have common hiring employers and not graduating departments. The results may be different if applicants had common graduating departments that were separately classified from hiring employers. I test this below:

### Experiment 4: Cluster by departments, where graduating departments have separate tiers from hiring employers

In [54]:
Random.seed!(0) # for reproducibility
num_applicants4 = length(department_list) # length(applicant_list)
@time est_obj4, est_alloc4 = SBM.doit(out, num_applicants4, length(department_list), nothing, [length(s) for s in sinks], NUMBER_OF_TYPES, numtotal, 500 * (NUMBER_OF_TYPES-2) + 1000)

8008.993554411652 



8×4 Matrix{Int32}:
 431  0  0  0
   0  0  0  0
   0  0  0  0
   0  0  0  0
 213  0  0  0
 203  0  0  0
  63  1  0  0
 608  1  0  0


8008.975736224549 8008.466085418052 8004.219928803282 8002.189430007194 8000.983258108322 7996.716026039997 7994.668311512265 7991.356028146611 7990.3818957863095 7989.511703548625 7988.217630895886 7986.865985558051 7985.892265174088 7985.65456135961 7983.87373215461 7981.257234952661 7980.085602961221 7979.104274158461 7975.619057188643 7973.820099755595 7970.940753093635 7967.992238386712 7967.8435220603715 7966.05625298886 7960.382509236767 7958.415392614245 7958.1702891362365 7951.375928755971 7949.103715385706 7947.011255149154 7945.668465345809 7942.548909369376 7939.306084905281 7936.405218738196 7934.766571164601 7933.60444698128 7931.790025538094 7930.62455710791 7928.642695524902 7926.369293093901 7925.393987540872 7924.225155560772 7920.837043362692 7919.649144351392 7918.48513772855 7916.325060908552 7915.935482302048 7912.467374885088 7911.305330263963 7910.941372666379 7910.840932777884 7910.495576227175 7906.9489739504115 7904.8957954881425 7899.84093067369 7897.446386

8×4 Matrix{Int32}:
 331   6  28   4
  37   0   2   0
  22   0   1   0
   0   0   0   0
 206   1   3   3
 183   6  14   0
  60   2   1   1
 547  17  35  10


7795.0272988735915 7794.683416673902 7792.179469359546 7788.402204392435 7784.565668396315 7780.66021835444 7776.943776631508 7771.279541020414 7770.157814400796 7767.603923398052 7766.201907140227 7765.076713044867 7762.97339268722 7760.497364813921 7759.422348698768 7759.2677071529115 7758.186394224243 7757.099033226171 7756.986756365268 7754.332623036081 7742.337686454804 7741.244488108744 7740.145631839562 7736.183353543593 7735.078992772607 7731.080778941366 7728.3626611323725 7728.102598972333 7724.055512286165 7721.530972507475 7717.575293482295 7716.471440426559 7713.895451226586 7712.128586675337 7709.525315286711 7708.696220195079 7706.12414514601 7704.653603214445 7700.497681043605 7698.34487739465 7697.250870810326 7696.151734149195 7693.367180743531 7691.9687910749035 7690.864608453823 7689.755452418403 7688.641385478688 7686.000481610758 7683.272133916024 7679.021209071345 7676.344275716257 7673.4933289166975 7672.37440483043 7671.283686067149 7668.377841322874 7667.2728

...

8×4 Matrix{Int32}:
  52  18   72   6
  31   8  122   0
  53   2   59   0
   0   0    0   8
  90   6   93  24
  69  34   98   2
  23   9   32   0
 233  44  246  86


6214.16555549087 6212.722223107801 6208.5549665364515 6207.730687651234 6207.116650859332 6198.671829328055 6198.038959820377 6197.386843552834 6196.056113921005 6190.529046181329 6189.571288976336 6189.10125158164 6188.413631311646 6188.077227847375 6184.925101995376 6184.500319661798 6181.985607832103 6180.330506322237 6179.453809286486 6175.552263525444 6174.601346901878 6173.6425654228115 6165.594146180018 6165.408181405451 6165.106602903475 6165.072298520712 6162.5276525396075 6160.498073121295 6159.733295050678 6158.842879151142 6157.944847597636 6157.394954804921 6155.840522001813 6154.942944651671 6150.592932019302 6149.68752368373 6143.877808304736 6143.200026990253 6142.396367713506 6141.483013007385 6140.8062174303 6136.145779137544 6135.163825110719 6131.7637202554215 6130.842300833599 6129.912695299561 6128.974779110227 6128.393151035016 6104.580130307864 6103.642108534899 6102.695517748703 6098.454747159845 6096.606204892064 6095.650909339345 6090.200787879803 6089.23664

8×4 Matrix{Int32}:
  32  20   53    4
  24  13  157    0
  53   0   67    0
   0   0    0    8
  64   5  114   30
  54  37  111    1
  16   9   38    1
 176  23  298  112


6007.786731203006 6006.7691535316335 6006.745266252869 6006.689766925042 6005.661450781671 6002.05592138426 5997.352567773564 5995.994138873328 5994.959406960614 5994.887104586877 5993.870848869428 5990.611546728405 5988.249922828516 5987.2223323822645 5986.281373016816 5980.228424496667 5979.189270933848 5978.837695860126 5976.038821655639 5974.987869187562 5973.924874311302 5972.849585479372 5971.469122459556 5971.284004434273 5970.899438138451 5967.895608529364 5966.43307432549 5965.31427599346 5963.874942446416 5962.389407317305 5961.257585337543 5960.405506735384 5959.269394360254 5958.9025929213885 5957.573713085806 5956.423960342667 5951.958432058703 5950.9858421961235 5950.280855287727 5943.736435359626 5942.051962203848 5940.976887856649 5940.60435095667 5939.0531490945295 5938.805003401346 5938.4323685169165 5937.343754096712 5936.241283959203 5930.3515230502035 5928.986739464801 5926.990741725589 5925.874088610314 5925.554792637622 5924.007551693722 5919.832943486169 5918.2

(5848.809182672581, Int32[4, 4, 2, 1, 2, 2, 1, 4, 1, 4  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [55]:
placement_rates4, counts4, sorted_allocation4, full_likelihood4 = SBM.get_allocation(est_alloc4, out, num_applicants4, NUMBER_OF_TYPES, numtotal, sort_tiers = 2)

(Int32[164 7 0 17; 80 72 0 0; … ; 39 15 0 10; 289 186 125 9], Int32[1125 2610 6345 6075; 1075 2494 6063 5805; … ; 250 580 1410 1350; 9650 22388 54426 52110], Int32[3, 3, 4, 2, 4, 4, 2, 3, 2, 3  …  8, 8, 8, 8, 8, 8, 8, 8, 8, 8], -7490.636572940705)

In [56]:
placement_rates4

8×4 Matrix{Int32}:
 164    7    0  17
  80   72    0   0
  38   23    0  21
   0    0    9   0
 116   59   33   5
 108   56    0  39
  39   15    0  10
 289  186  125   9

In [57]:
counts4

8×4 Matrix{Int32}:
 1125   2610   6345   6075
 1075   2494   6063   5805
 1775   4118  10011   9585
 5000  11600  28200  27000
 1625   3770   9165   8775
 1875   4350  10575  10125
  250    580   1410   1350
 9650  22388  54426  52110

In [58]:
placement_rates4 ./ counts4 # means

8×4 Matrix{Float64}:
 0.145778   0.00268199  0.0          0.00279835
 0.0744186  0.0288693   0.0          0.0
 0.0214085  0.00558524  0.0          0.00219092
 0.0        0.0         0.000319149  0.0
 0.0713846  0.0156499   0.00360065   0.000569801
 0.0576     0.0128736   0.0          0.00385185
 0.156      0.0258621   0.0          0.00740741
 0.0299482  0.00830802  0.0022967    0.000172712

In [59]:
full_likelihood4

-7490.636572940705

In [60]:
# since the hiring and graduating tiers are separate, the sorting will not work
# I sort the rows and columns by total number of placements
SBM.nice_table(placement_rates4, NUMBER_OF_TYPES, NUMBER_OF_SINKS, sink_labels)

┌───────────────────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │    164 │      7 │      0 │     17 │        188 │
│                Tier 2 │     80 │     72 │      0 │      0 │        152 │
│                Tier 3 │     38 │     23 │      0 │     21 │         82 │
│                Tier 4 │      0 │      0 │      9 │      0 │          9 │
│         Public Sector │    116 │     59 │     33 │      5 │        213 │
│        Private Sector │    108 │     56 │      0 │     39 │        203 │
│          Other Groups │     39 │     15 │      0 │     10 │         64 │
│ Teaching Universities │    289 │    186 │    125 │      9 │        609 │
│         Column Totals │    834 │    418 │    167 │    101 │       1520 │
└───────────────────────┴────────┴────────┴────────┴────────┴────────────┘
\begin{tabular}{rrrrrr}
 

In [61]:
for sorted_type in 1:NUMBER_OF_TYPES+NUMBER_OF_SINKS
    graduating_counter = 0
    hiring_counter = 0
    inst_hold_graduating = []
    inst_hold_hiring = []
    println("TYPE $sorted_type:")
    for (i, sbm_type) in enumerate(sorted_allocation4)
        if sbm_type == sorted_type
            if i <= length(department_list) # graduating departments
                push!(inst_hold_graduating, department_name_mapping[department_list[i]])
                graduating_counter += 1
            else # hiring departments
                if sorted_type <= NUMBER_OF_TYPES
                    push!(inst_hold_hiring, department_name_mapping[department_list[i - length(department_list)]])
                else
                    push!(inst_hold_hiring, sink_list[i - (2*length(department_list))])
                end
                hiring_counter += 1
            end
        end
    end
    println("  Graduating Departments:")
    for inst in sort(inst_hold_graduating)
        println("    ", inst)
    end
    println("  Total Graduating Departments: $graduating_counter")
    println()
    println("  Hiring Departments:")
    for inst in sort(inst_hold_hiring)
        println("    ", inst)
    end
    println("  Total Hiring Departments: $hiring_counter")
    println()
    println()
end

TYPE 1:
  Graduating Departments:
    Boston University - Economics
    Carnegie Mellon University - Tepper School of Business
    Columbia University - Economics
    Duke University - Economics
    Harvard University - Department of Economics
    Johns Hopkins University - Economics
    New York University - Economics
    Northwestern University - Economics
    Pennsylvania State University - Economics
    Princeton University - Economics
    Stanford University - Department of Economics
    University of British Columbia - Economics
    University of California Los Angeles (UCLA) - Economics
    University of California, Berkeley - Economics
    University of Chicago - Economics
    University of Mannheim - Department of Economics
    University of Maryland - Economics
    University of Michigan - Department of Economics
    University of Oxford - Economics
    University of Pennsylvania - Department of Economics
    University of Virginia - Economics
    University of Washington - D

The graduating and hiring tiers have slightly different compositions, and the placement rates are similar in structure to the case where graduating departments and hiring employers had the same tiers.

Note that candidates still have common tiers based on their graduating departments. In terms of the model, this means that candidates who graduated from the same departments have the same market value. This is still a stronger assumption than in the estimations which cluster candidates separately. A potential approach which does not have as strong of an assumption, but would not result in the exact matching produced by the estimations which cluster by candidates, would be to cluster candidates in groups based on other common factors such as CVs, where candidates with similar CVs have similar market values.

## Conclusion

This notebook explores several different approaches to estimating the tiers of candidates and employers in the job market for economics Ph.D. graduates. Different approaches lead to different classification structures; clustering by departments and employers only results in a placement matrix similar to previous Mapinator project results such as those in [estimate_model.ipynb](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_model.ipynb), whereas clustering by candidates results in a matching between hiring tiers and candidate tiers. The next steps would be to further establish the robustness of these approaches, estimate market parameters such as the market values of candidates, and use these estimates to study counterfactuals regarding the design of the market.

## Works Cited

Partly based on the citations in Peters & Yu (2023).

Chade, H. et al. (2017). "Sorting through Search and Matching Models in Economics." *Journal of Economic Literature*, 55(2), 493-544.

Karrer, B. and Newman, M. E. J. (2011). "Stochastic blockmodels and community structure in networks." *Physical Review E*, 83(1).

Peixoto, T. (2014). "Efficient Monte Carlo and greedy heuristic for the inference of stochastic block models." *Physical Review E*, 89(1).

Peters, M. and Yu, J. (2023). "The Mapinator Classification." (Work in progress). Retrieved 21 April 2024, from https://montoya.econ.ubc.ca/papers/markets/markets.pdf

Wang, Y. X. R. and Bickel, P. J. (2017). "Likelihood-Based Model Selection for Stochastic Block Models." *The Annals of Statistics*, 45(2), 500-528.